- transaction hash value

In [1]:
import os
from pyspark.sql.functions import col, lit, when, from_json
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StringType, IntegerType, ArrayType
import pyspark
from ast import literal_eval
from functools import reduce
from pyspark.sql.functions import col, lit, when, from_json
from graphframes import *
from delta import *
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import Row
from glob import glob
import networkx as nx
import pyvis
from pyvis.network import Network
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.types import *

In [2]:
config = pyspark.SparkConf().setAll([
    ('spark.driver.extraJavaOptions', '-Djava.io.tmpdir=/home/ubuntu/notebook/tmp_spark_files'),
    ('spark.executor.memory', '80g'), 
    ('spark.executor.cores', '6'),
    ('spark.cores.max', '30'), 
    ('spark.driver.memory', '20g'),
    ('spark.executor.instances', '5'),
    ('spark.worker.cleanup.enabled', 'true'),
    ('spark.worker.cleanup.interval', '60'),
    ('spark.worker.cleanup.appDataTtl', '60'),
    ('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.2'),
    ('spark.mongodb.output.writeConcern.wTimeoutMS', '120000'),
    ('spark.mongodb.output.writeConcern.socketTimeoutMS', '120000'),
    ('spark.mongodb.output.writeConcern.connectTimeoutMS', '120000')
])

In [3]:
spark = SparkSession \
    .builder \
    .config(conf=config) \
    .appName("MongoDB-query-network") \
    .master("spark://172.23.149.210:7077") \
    .getOrCreate()

22/09/07 22:38:28 WARN Utils: Your hostname, cardano-druid resolves to a loopback address: 127.0.0.1; using 172.23.149.210 instead (on interface ens3)
22/09/07 22:38:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.mongodb.spark#mongo-spark-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2d54fd72-aa74-4d77-9d82-d8d97c84a9d9;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector;10.0.2 in central
	found org.mongodb#mongodb-driver-sync;4.5.1 in central
	[4.5.1] org.mongodb#mongodb-driver-sync;[4.5.0,4.5.99)
	found org.mongodb#bson;4.5.1 in central
	found org.mongodb#mongodb-driver-core;4.5.1 in central
:: resolution report :: resolve 2531ms :: artifacts dl 7ms
	:: modules in use:
	org.mongodb#bson;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-core;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-sync;4.5.1 from central in [default]
	org.mongodb.spark#mongo-spark-connector;10.0.2 from central in [default]
	---------------------------------------------------------------------
	|                  |  

## Query: put together transaction in and transaction out

Tables needed:
- tx
- tx_in
- tx_out

### schemas & csv tables (not needed?)

### Import tables from MongoDB

In [4]:
tx = spark.read.format("mongodb") \
 .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .option('spark.mongodb.database', 'cardano') \
   .option('spark.mongodb.collection', 'node.public.tx') \
 .option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
 .option('spark.mongodb.change.stream.publish.full.document.only','true') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .load()

In [5]:
# massive table, 116.3M documents! Would be interesting to execute the Bokeh plot result and see what happens...
tx_in = spark.read.format("mongodb") \
 .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .option('spark.mongodb.database', 'cardano') \
   .option('spark.mongodb.collection', 'node.public.tx_in') \
 .option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
 .option('spark.mongodb.change.stream.publish.full.document.only','true') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .load()

In [6]:
tx_out = spark.read.format("mongodb") \
 .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .option('spark.mongodb.database', 'cardano') \
   .option('spark.mongodb.collection', 'node.public.tx_out') \
 .option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
 .option('spark.mongodb.change.stream.publish.full.document.only','true') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .load()

In [7]:
tx.createOrReplaceTempView("tx")
tx_in.createOrReplaceTempView("tx_in")
tx_out.createOrReplaceTempView("tx_out")

### Transaction outputs for specified transaction hash:

First, join the tx_out and tx tables on the respective transaction ids

In [8]:
qr_out = " SELECT tx_out.*, \
       tx_out.value \
FROM   tx_out \
       INNER JOIN tx \
               ON tx_out.tx_id = tx.id"

In [9]:
qr3 = " \
SELECT qr_out.id, \
       qr_out.address AS address_output, \
       tx.HASH \
FROM   qr_out \
       INNER JOIN tx \
               ON tx.id = qr_out.tx_id"

In [11]:
qr_in = " \
SELECT tx_out.*, \
       tx.HASH AS tx_hash \
FROM   tx_out \
       INNER JOIN tx_in \
               ON tx_out.tx_id = tx_in.tx_out_id \
       INNER JOIN tx \
               ON tx.id = tx_in.tx_in_id \
                  AND tx_in.tx_out_index = tx_out.index "

In [12]:
spark.sql(qr_out).createOrReplaceTempView("qr_out")
spark.sql(qr3).createOrReplaceTempView("qr3")
spark.sql(qr_in).createOrReplaceTempView("qr_in")

__Materialize the results and store them in MongoDB Silver db:__

In [13]:
qr_out_table = spark.sql(qr_out)
qr3_table = spark.sql(qr3)
qr_in_table = spark.sql(qr_in)

In [24]:
## Write in the GOLD db, collection smart_contract_calls
qr_out_table.write.format("mongodb") \
 .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("overwrite") \
    .option('spark.mongodb.database', 'cardano_silver') \
   .option('spark.mongodb.collection', 'qr_out_table') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

22/09/07 18:34:01 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


In [25]:
## Write in the GOLD db, collection smart_contract_calls
qr3_table.write.format("mongodb") \
 .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("overwrite") \
    .option('spark.mongodb.database', 'cardano_silver') \
   .option('spark.mongodb.collection', 'qr3_table') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

22/09/07 19:11:13 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


In [ ]:
## Write in the GOLD db, collection smart_contract_calls
qr_in_table.write.format("mongodb") \
 .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("overwrite") \
    .option('spark.mongodb.database', 'cardano_silver') \
   .option('spark.mongodb.collection', 'qr_in_table') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

22/09/07 22:43:19 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


joined on data_hash and hash (which both represent the hash identifier of the transaction)

In [17]:
query_join_out_in = " \
SELECT qr_in.address          AS input, \
       qr3.address_output     AS output, \
       qr_in.tx_hash, \
         qr_in.VALUE / 1000000 AS ADA_value \
FROM   qr_in \
       LEFT JOIN qr3 \
              ON qr_in.tx_hash = qr3.HASH \
WHERE  qr_in.address != qr3.address_output"

In [18]:
spark.sql(query_join_out_in).createOrReplaceTempView("query_join_out_in")

In [19]:
result = spark.sql(query_join_out_in)

In [20]:
## Write in the GOLD db, collection smart_contract_calls
result.write.format("mongodb") \
 .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .mode("overwrite") \
    .option('spark.mongodb.database', 'cardano_silver') \
   .option('spark.mongodb.collection', 'transaction_network') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .save()

22/09/06 20:59:32 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
22/09/06 21:01:25 WARN TaskSetManager: Lost task 130.0 in stage 0.0 (TID 130) (172.23.149.210 executor 0): com.mongodb.MongoSocketReadException: Prematurely reached end of stream
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:112)
	at com.mongodb.internal.connection.SocketStream.read(SocketStream.java:131)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveResponseBuffers(InternalStreamConnection.java:718)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveMessageWithAdditionalTimeout(InternalStreamConnection.java:576)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveCommandMessageResponse(InternalStreamConnection.java:415)
	at com.mongodb.internal.connection.InternalStreamConnection.sendAndReceive(InternalStreamConnection.java:342)
	at com.mongodb.internal.connection.Usa

22/09/06 21:01:55 WARN TaskSetManager: Lost task 136.0 in stage 0.0 (TID 136) (172.23.149.210 executor 1): com.mongodb.MongoSocketReadException: Exception receiving message
	at com.mongodb.internal.connection.InternalStreamConnection.translateReadException(InternalStreamConnection.java:707)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveMessageWithAdditionalTimeout(InternalStreamConnection.java:579)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveCommandMessageResponse(InternalStreamConnection.java:415)
	at com.mongodb.internal.connection.InternalStreamConnection.sendAndReceive(InternalStreamConnection.java:342)
	at com.mongodb.internal.connection.UsageTrackingInternalConnection.sendAndReceive(UsageTrackingInternalConnection.java:116)
	at com.mongodb.internal.connection.DefaultConnectionPool$PooledConnection.sendAndReceive(DefaultConnectionPool.java:643)
	at com.mongodb.internal.connection.CommandProtocolImpl.execute(CommandProtocolImpl.java:71)

22/09/06 21:02:25 WARN TaskSetManager: Lost task 136.1 in stage 0.0 (TID 147) (172.23.149.210 executor 1): com.mongodb.MongoTimeoutException: Timed out after 30000 ms while waiting to connect. Client view of cluster state is {type=UNKNOWN, servers=[{address=172.23.149.210:27017, type=UNKNOWN, state=CONNECTING, exception={com.mongodb.MongoSocketOpenException: Exception opening socket}, caused by {java.net.ConnectException: Connection refused (Connection refused)}}]
	at com.mongodb.internal.connection.BaseCluster.getDescription(BaseCluster.java:180)
	at com.mongodb.internal.connection.SingleServerCluster.getDescription(SingleServerCluster.java:44)
	at com.mongodb.client.internal.MongoClientDelegate.getConnectedClusterDescription(MongoClientDelegate.java:144)
	at com.mongodb.client.internal.MongoClientDelegate.createClientSession(MongoClientDelegate.java:101)
	at com.mongodb.client.internal.MongoClientDelegate$DelegateOperationExecutor.getClientSession(MongoClientDelegate.java:291)
	at co

Py4JJavaError: An error occurred while calling o122.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 133 in stage 0.0 failed 4 times, most recent failure: Lost task 133.3 in stage 0.0 (TID 168) (172.23.149.210 executor 0): com.mongodb.MongoTimeoutException: Timed out after 30000 ms while waiting to connect. Client view of cluster state is {type=UNKNOWN, servers=[{address=172.23.149.210:27017, type=UNKNOWN, state=CONNECTING, exception={com.mongodb.MongoSocketOpenException: Exception opening socket}, caused by {java.net.ConnectException: Connection refused (Connection refused)}}]
	at com.mongodb.internal.connection.BaseCluster.getDescription(BaseCluster.java:180)
	at com.mongodb.internal.connection.SingleServerCluster.getDescription(SingleServerCluster.java:44)
	at com.mongodb.client.internal.MongoClientDelegate.getConnectedClusterDescription(MongoClientDelegate.java:144)
	at com.mongodb.client.internal.MongoClientDelegate.createClientSession(MongoClientDel

In [ ]:
spark.stop()

22/09/06 21:02:57 WARN TaskSetManager: Lost task 137.3 in stage 0.0 (TID 177) (172.23.149.210 executor 0): TaskKilled (Stage cancelled)
22/09/06 21:02:57 WARN TaskSetManager: Lost task 138.1 in stage 0.0 (TID 172) (172.23.149.210 executor 1): TaskKilled (Stage cancelled)
22/09/06 21:02:57 WARN TaskSetManager: Lost task 123.3 in stage 0.0 (TID 173) (172.23.149.210 executor 1): TaskKilled (Stage cancelled)
22/09/06 21:02:57 WARN TaskSetManager: Lost task 140.3 in stage 0.0 (TID 178) (172.23.149.210 executor 0): TaskKilled (Stage cancelled)
22/09/06 21:02:57 WARN TaskSetManager: Lost task 141.3 in stage 0.0 (TID 179) (172.23.149.210 executor 1): TaskKilled (Stage cancelled)
22/09/06 21:02:57 WARN TaskSetManager: Lost task 124.3 in stage 0.0 (TID 170) (172.23.149.210 executor 1): TaskKilled (Stage cancelled)
22/09/06 21:02:57 WARN TaskSetManager: Lost task 139.3 in stage 0.0 (TID 180) (172.23.149.210 executor 0): TaskKilled (Stage cancelled)
22/09/06 21:02:57 WARN TaskSetManager: Lost task

# network creation

In [2]:
df = pd.read_csv('/home/ubuntu/notebook/query_results/network_query_spark/part-00000-aaa30817-0114-4a88-a6bb-b194614eaa65-c000.csv')

### Approach 1: Pyvis

In [3]:
net = Network(notebook = True, directed = True)

In [4]:
from pyvis.network import Network
import pandas as pd

df_net = Network(height='750px', width='100%', bgcolor='#222222', font_color='white')

# set the physics layout of the network
df_net.barnes_hut()

sources = df['input']
targets = df['output']
weights = df['ADA_value']

edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    df_net.add_node(src, src, title=src)
    df_net.add_node(dst, dst, title=dst)
    df_net.add_edge(src, dst, value=w)

neighbor_map = df_net.get_adj_list()

# add neighbor data to node hover data
for node in df_net.nodes:
    node['title'] += ' Neighbors:<br>' + '<br>'.join(neighbor_map[node['id']])
    node['value'] = len(neighbor_map[node['id']])

df_net.show('cardano_network.html')

)07=                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

### Approach 2: NetworkX and Bokeh

- objective: build a network of transactions (tx) between nodes (address)
- needed tables: tx_out, tx_in, tx

- tx_in columns:
    - tx_out_id -> The Tx table index of the transaction that contains this transaction input
    - tx_in_id -> The Tx table index of the transaction that contains this transaction output
- tx columns:
    - id -> which transaction contains the 
- tx_out columns:
    - address: from which address the transaction comes (who sent the ADA?)